In [1]:
import numpy as np
import keras
from matplotlib import pyplot as plt
import os

Using TensorFlow backend.


In [569]:
inshape = (25,43,)
stride=1
dilation=2
num_filters=19
kernel_size=9
pad = 'causal'
a = keras.layers.Input(inshape)
b = keras.layers.Dense(48)(a)
c = keras.layers.Conv1D(filters=num_filters, kernel_size=kernel_size, strides=stride, padding=pad, dilation_rate=dilation)(b)
#d = keras.layers.Input((8,32,))
#e = keras.layers.Dense(30)(d)
#f = keras.layers.add([e,c])

model = keras.models.Model(inputs=a, outputs=c)

In [570]:
x = np.random.random(inshape)
x = x[np.newaxis,...]
y = model.predict(x)
#model.layers[2].get_weights()[0].shape
y.flatten

<function ndarray.flatten>

In [571]:
denseA = model.layers[1].get_weights()[0]
denseb = model.layers[1].get_weights()[1]
y1 = (x[0,:,:]@denseA)+denseb
filters = model.layers[2].get_weights()[0]
bias = model.layers[2].get_weights()[1]
# for "same" padding,
# if kernel size is odd, pad by (nk-1)/2 on either side
# of even, 1 less on top
# for 'causal' padding, pad all on top [dilation_rate * (kernel_shape[0] - 1)]
(in_height,in_width) = model.layers[2].input_shape[1:]
out_width  = num_filters #int(np.ceil(float(in_width)))

if pad is 'causal':
    out_height = int(np.ceil(float(in_height) / float(stride)))

    pad_along_height = dilation*(kernel_size-1)
    pad_top = pad_along_height
    pad_bottom = 0
elif pad is 'same':
    out_height = int(np.ceil(float(in_height) / float(stride)))

    pad_along_height = max((out_height - 1) * stride*dilation +
                        kernel_size - in_height, 0)
    pad_top = int(pad_along_height // 2)
    pad_bottom = int(pad_along_height - pad_top)
elif pad is 'valid':
    out_height =int(np.floor(((in_height-filter_height)/(stride*dilation)+1)))
    pad_top=0
    pad_bottom=0
padded_in_height = in_height + pad_top + pad_bottom

#pad_left = pad_along_width // 2
#pad_right = pad_along_width - pad_left

y1 = np.concatenate((np.zeros((pad_top,48)),y1,np.zeros((pad_bottom,48))),0)
#y1 = np.concatenate((np.zeros((11,3)),y1),1)
#y1.shape
model.layers[2].get_weights()[0].shape
#(kernel_size-1)*in_width*out_width + (in_width-1)*in_width

(9, 48, 19)

In [584]:
#filters[0][47,29]
y2 = convolve(y1,filters,out_height,out_width,kernel_size,in_width,padded_in_height,stride,dilation)
#y2 = y2 + bias

In [585]:
np.mean(np.abs(y.flatten(order='C')-y2.flatten(order='C')))
#in_width*out_width*(kernel_size-1)

9.087181571735171e-08

In [583]:
def convolve(x,kernel,out_height,out_width,kernel_size,in_width,padded_in_height,stride,dilation):
    x = x.flatten(order='C')
    kernel = kernel.flatten(order='C')
    out_size = out_height*out_width
    output = np.zeros(out_size)
   
    for p in range(out_height):
        outrowidx = p*out_width
        for k in range(out_width):
            for z in range(kernel_size):
                kernelidx = z*in_width*out_width
                for q in range(in_width):
                    inheightidx = q*out_width
                    output[outrowidx+k] += kernel[kernelidx+ inheightidx+ k]*x[(p*stride+ z*dilation)*in_width+q];
            output[outrowidx+k] += bias[k]
    
#     output = np.zeros((out_height,out_width))
#     output_shape = output.shape
    
#     for p in range(out_height):
#         for k in range(out_width):
#             for z in range(kernel_size):
#                 for q in range(in_width):
#                     output[p,k] += filters[z][q,k]*x[p*stride+z*dilation,q]
#             output[x2,k] += bias[k]

#    for x2 in range(output_shape[0]):
#        for k in range(output_shape[1]):
#            for z in range(kernel_size):
#                for q in range(x.shape[1]):
#                    output[x2,k] += filters[z][q,k]*x[x2*stride+z*dilation,q]
#            output[x2,k] += bias[k]
    return output

In [186]:
model.layers[2].get_weights()[0].shape

(48, 28)

In [257]:
model.layers[1].output_shape

(None, 25, 48)

In [446]:
np.zeros(5)

array([0., 0., 0., 0., 0.])

In [553]:
kernel_size=9
in_width=48
filters=19
b = np.random.random((kernel_size,in_width,filters))
a = b.flatten(order='C')
idx = (4,17,2)
b[idx] - a[idx[0] * (in_width * filters) + (idx[1] * filters) + idx[2]]

0.0

In [ ]:
#out_height = ceil(float(in_height) / float(strides[1]))
#out_width  = ceil(float(in_width) / float(strides[2]))

#pad_along_height = max((out_height - 1) * strides[1] +
#                    filter_height - in_height, 0)
#pad_along_width = max((out_width - 1) * strides[2] +
#                   filter_width - in_width, 0)
#pad_top = pad_along_height // 2
#pad_bottom = pad_along_height - pad_top
#pad_left = pad_along_width // 2
#pad_right = pad_along_width - pad_left


import pydot
#graph = pydot.graph_from_dot_data(str())
graph = keras.utils.vis_utils.model_to_dot(model)
print(graph.get_nodes()[1])

In [ ]:
def array2c(array,name):
    shp = array.shape
    count = 0
    if len(shp) is 1:
        s = 'float ' + name + '[' + str(shp[0]) + '] = \n{'
        for i in range(shp[0]):
            s += "{:.16f}".format(array[i]) + ','
            count += 1
            if (count)%3 is 0:
                s += '\n'
        s += '}; \n'
        return s    
    elif len(shp) is 2:
        s = 'float ' + name + '[' + str(shp[0]) + '][' + str(shp[1]) + '] = \n{'
        for row in range(shp[0]):
            s += '{'
            for col in range(shp[1]):
                s += "{:.16f}".format(array[row,col]) 
                if col < shp[1]:
                    s += ','
                count += 1
                if (count)%3 is 0:
                    s += '\n'
            s += '}'
            if row<shp[0]:
                s += ','
            s += '\n'
        s += '}; \n'
        return s
                
    elif len(shp) is 3:
        s = 'float ' + name + '[' + str(shp[0]) + '][' + str(shp[1]) + '][' + str(shp[2]) + '] = \n{'
        for filters in range(shp[0]):
            s += '{'
            for row in range(shp[1]):
                s += '{'
                for col in range(shp[0]):
                    s += "{:.16f}".format(array[filters,row,col]) 
                    if col < shp[1]:
                        s += ','
                    count += 1
                    if (count)%3 is 0:
                        s += '\n'
                s += '}'
                if row<shp[1]:
                    s += ','
                s += '\n'
            s += '}'
            if filters<shp[0]:
                s += ','
        s += '}; \n'
        return s